In [1]:
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws
import pandas as pd
import time
from datetime import date

# Replace these values with your actual API credentials
client_id = "PP860PBEU0-100"
secret_key = "JC501DPXH7"
redirect_uri = "https://google.com"
response_type = "code"  
state = "sample_state"
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri,
    response_type=response_type
)
response = session.generate_authcode()
print(response)

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=PP860PBEU0-100&redirect_uri=https%3A%2F%2Fgoogle.com&response_type=code&state=None


In [2]:
auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE2OTkzNzc3NDksImV4cCI6MTY5OTQwNzc0OSwibmJmIjoxNjk5Mzc3MTQ5LCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJYUzk4MzY4Iiwib21zIjoiSzEiLCJoc21fa2V5IjoiYmIyM2YxMjg2ODFiNGE3OWM2NmU2MTM4MDdkNjZhNGM5NmE5MjUzNDZhYjhiZWRkYzQ0MDdmMzYiLCJub25jZSI6IiIsImFwcF9pZCI6IlBQODYwUEJFVTAiLCJ1dWlkIjoiOTRhYTcxYWJiMjU2NDA0MWFkY2Y4MDJlZTdkNGU5ODgiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.qFRD8Xg77fROXwY9d6tJPvpLQQsebVmTIWl8W9M__DE"

In [3]:
grant_type = "authorization_code"
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)
session.set_token(auth_code)
response = session.generate_token()
print(response)

{'s': 'ok', 'code': 200, 'message': '', 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTkzNzc4NjMsImV4cCI6MTY5OTQwMzQyMywibmJmIjoxNjk5Mzc3ODYzLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbFNuTEh4R05kYVlkSWI5YUlpbVNucFo5MDJ5SmNYbnh3X1NoWHJ2RFQycVRyeWhNRlZLX3VFbUhrQ0twZjlpcUZ1d2RhZmpRNnA4b0FNU0FZcVJ6ZFVieTlFdml5MXd2a2lpOEdkOU5nS1pLWW5wRT0iLCJkaXNwbGF5X25hbWUiOiJTQU5TS0FSIFNSSVZBU1RBVkEiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiYjIzZjEyODY4MWI0YTc5YzY2ZTYxMzgwN2Q2NmE0Yzk2YTkyNTM0NmFiOGJlZGRjNDQwN2YzNiIsImZ5X2lkIjoiWFM5ODM2OCIsImFwcFR5cGUiOjEwMCwicG9hX2ZsYWciOiJOIn0.wRahiob2ggU2eL_qWBvhgetpVY_DssQ9m1vH2mdnvMA', 'refresh_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTkzNzc4NjMsImV4cCI6MTcwMDYxMzAyMywibmJmIjoxNjk5Mzc3ODYzLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6InJlZnJlc2hfdG9rZW4iLCJhdF9oYXNoIjoiZ0FBQUFBQ

In [41]:
df=pd.DataFrame()
df=response
tk=df['access_token']
print(tk)

eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTkzNzc4NjMsImV4cCI6MTY5OTQwMzQyMywibmJmIjoxNjk5Mzc3ODYzLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbFNuTEh4R05kYVlkSWI5YUlpbVNucFo5MDJ5SmNYbnh3X1NoWHJ2RFQycVRyeWhNRlZLX3VFbUhrQ0twZjlpcUZ1d2RhZmpRNnA4b0FNU0FZcVJ6ZFVieTlFdml5MXd2a2lpOEdkOU5nS1pLWW5wRT0iLCJkaXNwbGF5X25hbWUiOiJTQU5TS0FSIFNSSVZBU1RBVkEiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiYjIzZjEyODY4MWI0YTc5YzY2ZTYxMzgwN2Q2NmE0Yzk2YTkyNTM0NmFiOGJlZGRjNDQwN2YzNiIsImZ5X2lkIjoiWFM5ODM2OCIsImFwcFR5cGUiOjEwMCwicG9hX2ZsYWciOiJOIn0.wRahiob2ggU2eL_qWBvhgetpVY_DssQ9m1vH2mdnvMA


In [40]:
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=tk, log_path="")

In [6]:
def signal_generator(df):
    Open = df.open.iloc[-1]
    Close = df.close.iloc[-1]
    previous_Open = df.open.iloc[-2]
    previous_Close = df.close.iloc[-2]
    
    # red candle
    if (Open>Close):
        return 1

    # green candle
    elif (Open<Close):
        return 2
    
    # No clear pattern
    else:
        return 0


In [7]:
call=input(str("Enter the symbol for call :"))
put=input(str("Enter the symbol for put :"))
calltight=int(input("Enter qty for tight lots for call option:  "))
callspare=int(input("Enter qty for spare lots for call option:  "))
puttight=int(input("Enter qty for tight lots for put option:  "))
putspare=int(input("Enter qty for spare lots for put option:  "))

Enter the symbol for call :NSE:BANKNIFTY23N0843100CE
Enter the symbol for put :NSE:BANKNIFTY23N0843600PE
Enter qty for tight lots for call option:  5
Enter qty for spare lots for call option:  5
Enter qty for tight lots for put option:  5
Enter qty for spare lots for put option:  5


In [26]:
date=date.today()
print(date)

2023-11-07


In [42]:
def get_datacall():
    data = {
    "symbol":call,
    "resolution":"5",
    "date_format":"1",
    "range_from":"2023-11-07",
    "range_to":"2023-11-07",
    "cont_flag":"1"}

    hdata=fyers.history(data=data)
    df = pd.DataFrame.from_dict(hdata['candles'])
    cols = ['datetime','open','high','low','close','volume']
    df.columns=cols
    df['datetime']= pd.to_datetime(df['datetime'],unit="s")
    df['datetime'] = df['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    
    signal = []
    signal.append(0)
    for i in range(1,len(df)):
        dataF = df[i-1:i+1]
        signal.append(signal_generator(dataF))
    #signal_generator(data)
    df["signal"] = signal
    
    goodpair = pd.DataFrame(columns=['Date', 'Time', 'HC', 'LC', 'ET', 'ES'])
    
    for i in range(1,len(df)):
        h=0
        l=0
        t=0
        signalfirst = df.signal.iloc[i]
        signalsecond = df.signal.iloc[i-1]
        presenthigh = df.high.iloc[i]
        previoushigh = df.high.iloc[i-1]
        presentlow = df.low.iloc[i]
        previouslow = df.low.iloc[i-1]
        previoustime = df.time.iloc[i-1]
        presenttime = df.time.iloc[i]
        previousdate = df.date.iloc[i-1]
        presentdate = df.date.iloc[i]
    
        if(signalfirst!=signalsecond):
            if(previoushigh>presenthigh and presentlow<previouslow):
                h = previoushigh
                l = presentlow
                t=1
            elif(previoushigh<presenthigh and presentlow>previouslow):
                h = presenthigh
                l = previouslow
                t=1
        if(abs(h-l)>=0 and abs(h-l)<=30 and t==1):
            goodpair.loc[len(goodpair)] = [previousdate, previoustime, h, l, 0, 0]
    global goodpairCE
    goodpairCE=goodpair

In [43]:
get_datacall()
goodpairCE

,Date,Time,HC,LC,ET,ES
0,2023-11-07,10:15:00,401.70,375.50,0,0
1,2023-11-07,10:55:00,430.00,410.80,0,0
2,2023-11-07,11:10:00,450.75,427.35,0,0
3,2023-11-07,11:15:00,443.70,427.35,0,0
4,2023-11-07,12:00:00,291.80,270.70,0,0
5,2023-11-07,12:15:00,297.40,275.00,0,0
6,2023-11-07,12:35:00,292.65,278.60,0,0
7,2023-11-07,13:05:00,313.40,295.00,0,0
8,2023-11-07,13:10:00,318.00,295.00,0,0
9,2023-11-07,13:15:00,318.00,292.25,0,0


In [44]:
def get_dataput():
    data = {
    "symbol":put,
    "resolution":"5",
    "date_format":"1",
    "range_from":"2023-11-07",
    "range_to":"2023-11-07",
    "cont_flag":"1"
    }

    hdata=fyers.history(data=data)
    df = pd.DataFrame.from_dict(hdata['candles'])
    cols = ['datetime','open','high','low','close','volume']
    df.columns=cols
    df['datetime']= pd.to_datetime(df['datetime'],unit="s")
    df['datetime'] = df['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    
    signal = []
    signal.append(0)
    for i in range(1,len(df)):
        dataF = df[i-1:i+1]
        signal.append(signal_generator(dataF))
    #signal_generator(data)
    df["signal"] = signal
    
    goodpair = pd.DataFrame(columns=['Date', 'Time', 'HP', 'LP', 'ET', 'ES'])
    
    for i in range(1,len(df)):
        h=0
        l=0
        t=0
        signalfirst = df.signal.iloc[i]
        signalsecond = df.signal.iloc[i-1]
        presenthigh = df.high.iloc[i]
        previoushigh = df.high.iloc[i-1]
        presentlow = df.low.iloc[i]
        previouslow = df.low.iloc[i-1]
        previoustime = df.time.iloc[i-1]
        presenttime = df.time.iloc[i]
        previousdate = df.date.iloc[i-1]
        presentdate = df.date.iloc[i]
    
        if(signalfirst!=signalsecond):
            if(previoushigh>presenthigh and presentlow<previouslow):
                h = previoushigh
                l = presentlow
                t=1
            elif(previoushigh<presenthigh and presentlow>previouslow):
                h = presenthigh
                l = previouslow
                t=1
        if(abs(h-l)>=0 and abs(h-l)<=30 and t==1):
            goodpair.loc[len(goodpair)] = [previousdate, previoustime, h, l, 0, 0]
    global goodpairPE
    goodpairPE=goodpair

In [45]:
get_dataput()
goodpairPE

,Date,Time,HP,LP,ET,ES
0,2023-11-07,10:15:00,239.50,214.05,0,0
1,2023-11-07,10:20:00,232.00,214.05,0,0
2,2023-11-07,11:05:00,195.45,172.80,0,0
3,2023-11-07,11:10:00,188.00,172.80,0,0
4,2023-11-07,12:00:00,336.05,308.40,0,0
5,2023-11-07,12:35:00,318.85,300.95,0,0
6,2023-11-07,13:05:00,288.75,269.50,0,0
7,2023-11-07,13:10:00,288.75,265.40,0,0
8,2023-11-07,13:15:00,293.75,265.40,0,0
9,2023-11-07,14:05:00,134.95,108.45,0,0


In [60]:
def onmessage(message):
    t = time.localtime()
    cmin = time.strftime("%M", t)
    csec = time.strftime("%S", t)
    if (int(cmin) % 5 == 0 and int(csec) <5):
        get_datacall()
        get_dataput()
        '''if (len(goodpairCE)>=1 and goodpairCE.Date.iloc[-1]==date.today()):
            global hc
            hc = goodpairCE.HC.iloc[-1]
            global lc
            lc = goodpairCE.LC.iloc[-1]
        if (len(goodpairPE)>=1 and goodpairPE.Date.iloc[-1]==date.today()):
            global hp
            hp = goodpairPE.HP.iloc[-1]
            global lp
            lp = goodpairPE.LP.iloc[-1]'''
    print("Response : ", message)
    if (len(goodpairCE)>=1 and goodpairCE.Date.iloc[-1]==date.today() and goodpairCE.ET.iloc[-1]==0):
        data_ct = {
            "symbol":call,
            "qty":calltight,
            "type":1,
            "side":1,
            "productType":"BO",
            "limitPrice":goodpairCE.HC.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairCE.LC.iloc[-1],
            "takeProfit":goodpairCE.HC.iloc[-1]+15
        }
        response_ct = fyers.place_order(data=data_ct)
        #print(response_ct)
        if response_ct['s']=='ok':
            print(f"entry tight slot for call {call}")
            goodpairCE.ET.iloc[-1]=1
        
    if (len(goodpairPE)>=1 and goodpairPE.Date.iloc[-1]==date.today() and goodpairPE.ET.iloc[-1]==0):
        data_pt = {
            "symbol":put,
            "qty":puttight,
            "type":1,
            "side":1,
            "productType":"BO",
            "limitPrice":goodpairPE.HP.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairPE.LP.iloc[-1],
            "takeProfit":goodpairPE.HP.iloc[-1]+15
        }
        response_pt = fyers.place_order(data=data_pt)
        if response_pt['s']=='ok':
            print(f"entry tight slot for put {put}")
            goodpairPE.ET.iloc[-1]=1
            
    if (len(goodpairCE)>=1 and goodpairCE.Date.iloc[-1]==date.today() and goodpairCE.ES.iloc[-1]==0):
        data_cs = {
            "symbol":call,
            "qty":callspare,
            "type":1,
            "side":1,
            "productType":"CO",
            "limitPrice":goodpairCE.HC.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairCE.LC.iloc[-1],
            "takeProfit":0
        }
        global response_cs
        response_cs = fyers.place_order(data=data_cs)
        if response_cs['s']=='ok':
            print(f"entry spare slot for call {call}")
            goodpairCE.ES.iloc[-1]=1
            global targetC
            targetC=goodpairCE.HC.iloc[-1]+15
            global slC
            slC=goodpairCE.LC.iloc[-1]
            
    if(goodpairCE.ES.iloc[-1]==1 and message['symbol']==call and message['ltp']>=targetC):
        targetC=targetC+15
        slC=targetC-30
        orderIdC = response_cs['id']
        data_mcs = {
            "id":orderIdC, 
            "type":1, 
            "limitPrice":slC,
            "qty":callspare
        }
        response_mcs = fyers.modify_order(data=data_mcs)
        if response_mcs['s']=='ok':
            print(f"spare slot trailed for call {call} at sl={slC}")
            
    if (len(goodpairPE)>=1 and goodpairPE.Date.iloc[-1]==date.today() and goodpairPE.ES.iloc[-1]==0):
        data_ps = {
            "symbol":put,
            "qty":putspare,
            "type":1,
            "side":1,
            "productType":"CO",
            "limitPrice":goodpairPE.HP.iloc[-1],
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            "stopLoss":goodpairPE.LP.iloc[-1],
            "takeProfit":0
        }
        global response_ps
        response_ps = fyers.place_order(data=data_ps)
        if response_ps['s']=='ok':
            print(f"entry spare slot for put {put}")
            goodpairPE.ES.iloc[-1]=1
            global targetP
            targetP=goodpairPE.HP.iloc[-1]+15
            global slP
            slP=goodpairPE.LP.iloc[-1]
            
    if(goodpairPE.ES.iloc[-1]==1 and message['symbol']==put and message['ltp']>=targetP):
        targetP=targetP+15
        slP=targetP-30
        orderIdP = response_ps['id']
        data_mps = {
            "id":orderIdP, 
            "type":1, 
            "limitPrice": slP,
            "qty":putspare
        }
        response_mps = fyers.modify_order(data=data_mps)
        if response_mps['s']=='ok':
            print(f"spare slot trailed for put {put} at sl={slP}")

def onerror(message):

    print("Error:", message)


def onclose(message):
  
    print("Connection closed:", message)


def onopen():
    data_type = "SymbolUpdate"
    symbols = [call,put]
    #print(len(goodpairPE))
    ofyers.subscribe(symbols=symbols, data_type=data_type)
    ofyers.keep_running()
access_token = client_id + ":" + tk

ofyers = data_ws.FyersDataSocket(
    access_token=access_token,
    log_path="",                     
    litemode=False,
    write_to_file=False,              
    reconnect=True,                 
    on_connect=onopen,               
    on_close=onclose,                
    on_error=onerror,               
    on_message=onmessage             
)
ofyers.connect()

Response :  {'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
Response :  {'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
Response :  {'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Response :  {'ltp': 669.65, 'vol_traded_today': 11554425, 'last_traded_time': 1699351199, 'exch_feed_time': 1699351200, 'bid_size': 120, 'ask_size': 45, 'bid_price': 664.45, 'ask_price': 670.0, 'last_traded_qty': 210, 'tot_buy_qty': 37200, 'tot_sell_qty': 4635, 'avg_trade_price': 378.6, 'low_price': 255.0, 'high_price': 705.75, 'lower_ckt': 0, 'upper_ckt': 0, 'open_price': 503.95, 'prev_close_price': 372.3, 'type': 'sf', 'symbol': 'NSE:BANKNIFTY23N0843100CE', 'ch': 297.35, 'chp': 79.8684}
Response :  {'ltp': 54.0, 'vol_traded_today': 94292010, 'last_traded_time': 1699351199, 'exch_feed_time': 1699351200, 'bid_size': 135, 'ask_size': 705, 'bid_price': 54.2, 'ask_price': 56.0, 'last_traded_qty': 15, 'tot_buy_qty': 87345, 'tot_sell_qty': 416025, 'avg_t